Wahlpflichtfach Künstliche Intelligenz II: Praktikum 

---

# 04 - Hyperparameter Optimization mit Tensorflow

Nachdem wir uns angeschaut haben, wie wir das Lernen verbessern können schauen wir uns nun an wie wir die Hyperparameter optimieren. 

In [ ]:
import tensorflow as tf

In [2]:
%matplotlib inline

In [ ]:
print(tf.__version__)

## Erstellen eines kleinen klassifikationsdaten

In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

iris = datasets.load_iris()
y = to_categorical(iris.target)
x = iris.data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

## Hyperparameter Optimization

Hyperparameter sind all die Parameter, die ein ML-Algorithmus nicht selbst lernt. Beispiel hierfür sind die Anzahl der Layer und jeweiligen Neuronen pro Layer (Netz Struktur). Weitere Hyperparameter sind:
* Optimizer oder Learning Rate
* Loss Funktion
* Anzahl der Epochen
* Batch Größe
* Regularization (Gewicht, Netz, Early Stopping)
* Pro Layer
  * Aktivierungsfunktion
  * Anzahl an Neuronen
  * Gewichtsinitialisierung
* Anzahl an Layern und Struktur (z. B.: Funnel, Bricks)

In [ ]:
from keras_tuner import HyperParameters, BayesianOptimization
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import (
    CategoricalAccuracy,
    Recall,
    Precision,
)

def build_model(hp: HyperParameters):
    model = Sequential()
    model.add(Input(shape=(4, )))

    for i in range(hp.Int("num_layers", min_value=1, max_value=5, step=1)):
        model.add(Dense(
            hp.Int(f'units_{i}', min_value=4, max_value=32, step=2),
            activation="relu"
        ))

        if hp.Boolean(f"dropout_{i}"):
            model.add(Dropout(rate=hp.Float(f"dropout_rate", min_value=0.1, max_value=0.7,
                                            step=0.1)))

    model.add(Dense(3, activation="softmax"))

    model.compile(
        optimizer=Adam(learning_rate=hp.Float("lr", min_value=1e-4, max_value=5e-4, step=1e-4)),
        loss=CategoricalCrossentropy(),
        metrics=[
            CategoricalAccuracy(),
            Recall(),
            Precision(),
        ]
    )

    return model

In [ ]:
from datetime import datetime
from pathlib import Path
from keras.callbacks import EarlyStopping

root_dir = Path.cwd()

tuner = BayesianOptimization(
    hypermodel=build_model,
    objective="val_categorical_accuracy",
    max_trials=10,
    executions_per_trial=2,
    directory=root_dir / "logs" / f"bayesian_{datetime.now().strftime('%Y.%m.%d-%H:%M:%S')}",
    project_name=f"bayesian_{datetime.now().strftime('%Y.%m.%d-%H:%M:%S')}_categorical_accuracy"
)

tuner.search(
    x_train, 
    y_train, 
    epochs=15, 
    validation_split=0.2,
    callbacks=[EarlyStopping(patience=3)]
)

### Das beste Modell selektieren

In [ ]:
best_model = tuner.get_best_models()[0]
loss, acc, rec, prec = best_model.evaluate(x_test, y_test)

---

Wahlpflichtach Künstliche Intelligenz II: Praktikum 